In [20]:
# mount my google drive
from google.colab import drive
drive.mount('/content/drive')

# change working directory 
%cd /content/drive/My Drive/Assignment2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Assignment2


In [21]:
# install deployment packages
!pip install flask
!pip install keras
!pip intall PIL

In [25]:
# Code for depolyment in Local PC 
from flask import Flask, request, jsonify
import keras
import numpy as np
from PIL import Image

# Load the pre-trained model
model = keras.models.load_model('cuisine_model.h5')

# Create a Flask application
app = Flask(__name__)

# Define a route that accepts POST requests with uploaded image files and returns the prediction
@app.route('/predict', methods=['POST'])
def predict():
    # Get the uploaded file
    file = request.files['file']
    # Convert the file to a PIL Image object
    img = Image.open(file.stream)
    # Resize the image to the size required by the model
    img = img.resize((224, 224))
    # Convert the image to a numpy array
    img_arr = np.array(img)
    # Convert the image from RGB format to BGR format (consistent with the pre-trained model)
    img_arr = img_arr[:, :, ::-1]
    # Scale the values of the image to be between 0 and 1 (consistent with the preprocessing of the pre-trained model)
    img_arr = img_arr.astype('float32') / 255.0
    # Add an extra dimension to the image to match the input shape of the model
    img_arr = np.expand_dims(img_arr, axis=0)
    # Use the pre-trained model for prediction
    pred = model.predict(img_arr)[0]
    # Create a dictionary to hold the prediction result
    result = {'Japanese': pred[0], 'Italian': pred[1], 'Chinese': pred[2], 'French': pred[3]}
    # If the maximum probability is less than 0.5, classify as "Other"
    if max(pred) < 0.5:
        result['Other'] = 1.0
    else:
        # Convert the prediction result to a dictionary and return it
        result = {'Japanese': pred[0], 'Italian': pred[1], 'Chinese': pred[2], 'French': pred[3]}
    return jsonify(result)

# Run the application
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
    

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# Install additional packages only deploy model in Colab Notbook
!pip install flask-ngrok
!pip install pyngrok

In [27]:
# Code for deployment in Colab by third website server
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import keras
import numpy as np
from PIL import Image

# Load the pre-trained model
model = keras.models.load_model('cuisine_model.h5')

# Create a Flask application
app = Flask(__name__)

# Set up ngrok and give authtoken (here is for ngrok account Rubing Wang)
from pyngrok import ngrok
ngrok.set_auth_token('2PTUDFWN0ZX1gvit7ALvL14qq4y_5p8ddyGn85qRXNvGq85a7')
#public_url = ngrok.connect(port=5000).public_url
public_url = ngrok.connect(addr="127.0.0.1:5000")
print(' * ngrok tunnel:', public_url)


# Define a route that accepts POST requests with uploaded image files and returns the prediction
@app.route('/predict', methods=['POST'])
def predict():
    # Get the uploaded file
    file = request.files['file']
    # Convert the file to a PIL Image object
    img = Image.open(file.stream)
    # Resize the image to the size required by the model
    img = img.resize((224, 224))
    # Convert the image to a numpy array
    img_arr = np.array(img)
    # Convert the image from RGB format to BGR format (consistent with the pre-trained model)
    img_arr = img_arr[:, :, ::-1]
    # Scale the values of the image to be between 0 and 1 (consistent with the preprocessing of the pre-trained model)
    img_arr = img_arr.astype('float32') / 255.0
    # Add an extra dimension to the image to match the input shape of the model
    img_arr = np.expand_dims(img_arr, axis=0)
    # Use the pre-trained model for prediction
    pred = model.predict(img_arr)[0]
    # Create a dictionary to hold the prediction result
    result = {'Japanese': pred[0], 'Italian': pred[1], 'Chinese': pred[2], 'French': pred[3]}
    # If the maximum probability is less than 0.5, classify as "Other"
    if max(pred) < 0.5:
        result['Other'] = 1.0
    else:
        # Convert the prediction result to a dictionary and return it
        result = {'Japanese': pred[0], 'Italian': pred[1], 'Chinese': pred[2], 'French': pred[3]}
    return jsonify(result)


# Run the application
if __name__ == '__main__':
    # Run the Flask application with ngrok
    run_with_ngrok(app)
    app.run()


 * ngrok tunnel: NgrokTunnel: "https://afee-34-83-73-92.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
